## memory 공유 확인 위한 dummy chain 생성

In [1]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [6]:
dummy_chain = dummy_prompt | model 

In [7]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [8]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 취소하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run fe98663a-199a-4811-97e7-32eb1dc9b7c9 not found for run 272b1ec9-d7ab-4f7b-923d-ee09c5b010c1. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ddadc334-cda2-4ae0-83bf-40cc0b23938a-0')

In [ ]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 취소하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

In [19]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 변경하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 27f97e00-2bd8-437c-9c98-a9b0f1f31573 not found for run 414bb4aa-89e4-45b1-b563-12c27c8c9bc2. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 169, 'total_tokens': 172}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0b352454-e795-485a-a26a-c38bada679c7-0')

## 실험 체인 생성

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "order_id:{order_id}"),
    ]
)

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [12]:
classify_chain = order_change_cancel_prompt | model

In [13]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_chain_with_memory = RunnableWithMessageHistory(
    classify_chain,
    get_session_history,
    input_messages_key="order_id",
    history_messages_key="chat_history",
)

In [15]:
response = classify_chain_with_memory.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 69269b3c-174f-49f6-aefd-d949896c37cd not found for run dde42eb9-d775-49bf-a2e0-76fa1eedffef. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 83, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-79530680-f4ca-4ccf-9208-2908dd63632c-0')

In [18]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ddadc334-cda2-4ae0-83bf-40cc0b23938a-0')]

In [20]:
response = classify_chain_with_memory.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 2eb3a18c-91d1-4301-8ae2-b6c28f6083c5 not found for run 00408ca8-c8f7-4591-9a8e-926bf6581083. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 102, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ec19f61f-1e55-46c6-88f5-6b057bfe8f6c-0')

classify_chain_with_memory의 출력은 저장이 안 되네?

In [21]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ddadc334-cda2-4ae0-83bf-40cc0b23938a-0'),
 HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 169, 'total_tokens': 172}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0b352454-e795-485a-a26a-c38bada679c7-0')]

## order_id 주어졌을 때 처리 체인

In [25]:
response = classify_chain_with_memory.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run a5270db7-107f-4bab-a48c-3506bc7e2053 not found for run 9262573e-ab0b-4e2f-bd27-3d2c314a8eed. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


In [35]:
def fetch_recent_orders(dict):
    print(dict)
    return "produdct: 백설기 quantity: 2 price: 13,000"

In [41]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the check_purpose and queried_result.
            create a message to show the queried_result and ask for final confirmation considering check_purpose.
            The response should be generated in Korean.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "check_purpose:{check_purpose}\nqueried_result:{queried_result}")
    ]
)

In [42]:
generate_confirm_message_chain = generate_confirm_message_prompt | model

In [43]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

test_chain1 = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | generate_confirm_message_chain
test_chain1

{
  order_id: RunnablePassthrough(),
  check_purpose: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
                   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
                 }), config={'run_name': 'insert_history'})
                 | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from 

In [44]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ddadc334-cda2-4ae0-83bf-40cc0b23938a-0'),
 HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 169, 'total_tokens': 172}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0b352454-e795-485a-a26a-c38bada679c7-0')]

In [45]:
test_chain1.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 28438fb1-5617-465c-84f5-46f8bb2e1502 not found for run 4280a95f-4268-45d5-be10-30dff4d1c810. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 내용을 확인해보실래요?\n\n상품: 백설기\n수량: 2개\n가격: 13,000원\n\n주문 변경을 원하시나요? 최종 확인 부탁드립니다.', response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 185, 'total_tokens': 251}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-215495d7-2c89-4565-8cdd-cdb7b4a3bbf0-0')

---

In [46]:
def debug(dict):
    return dict

In [47]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

try_fix_chain = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | RunnableLambda(debug)
try_fix_chain

{
  order_id: RunnablePassthrough(),
  check_purpose: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
                   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
                 }), config={'run_name': 'insert_history'})
                 | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from 

In [48]:
try_fix_chain.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 47b42aa4-32f2-4813-8301-ba207c062577 not found for run c8fac5b0-f8c0-4253-8c93-9cfe33b3900b. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


{'order_id': {'order_id': 3},
 'check_purpose': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 102, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4c373960-6f10-4750-b92c-a178700cf3d1-0'),
 'queried_result': 'produdct: 백설기 quantity: 2 price: 13,000'}

In [50]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()

In [51]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

try_fix_chain = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory | str_parser,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | RunnableLambda(debug)
try_fix_chain

{
  order_id: RunnablePassthrough(),
  check_purpose: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
                   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
                 }), config={'run_name': 'insert_history'})
                 | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from 

In [52]:
try_fix_chain.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 1a4e0759-d493-482a-b938-614b5d870405 not found for run b503f226-03e9-4427-8cb0-c5268147188f. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


{'order_id': {'order_id': 3},
 'check_purpose': '주문 변경',
 'queried_result': 'produdct: 백설기 quantity: 2 price: 13,000'}

In [53]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

try_fix_chain = {
    "order_id": lambda input: input["order_id"],
    "check_purpose": classify_chain_with_memory | str_parser,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | RunnableLambda(debug)
try_fix_chain

{
  order_id: RunnableLambda(...),
  check_purpose: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
                   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
                 }), config={'run_name': 'insert_history'})
                 | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from th

In [54]:
try_fix_chain.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 0e9721e1-48b9-4369-9a0b-439b8f126cde not found for run 5af93389-9543-4111-b90c-d1f3b21dee76. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


{'order_id': 3,
 'check_purpose': '주문 변경',
 'queried_result': 'produdct: 백설기 quantity: 2 price: 13,000'}

---

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

test_chain1 = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | generate_confirm_message_chain
test_chain1

In [56]:
test_chain2 = (
    classify_chain_with_memory | 
    RunnablePassthrough.assign(check_purpose=classify_chain_with_memory) | 
    RunnablePassthrough.assign(queried_result=RunnableLambda(fetch_recent_orders)) |
    generate_confirm_message_chain
    )
test_chain2 

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(vari

In [57]:
test_chain2.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 94f73d1e-cd00-49dd-bce4-fb2967cf08bc not found for run 3bbec3cf-a86f-4ed7-bdc7-ba980409a60e. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AssertionError: The input to RunnablePassthrough.assign() must be a dict.

## 확인 요청 메시지 작성 이후 부분

In [ ]:
test_chain1.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 28438fb1-5617-465c-84f5-46f8bb2e1502 not found for run 4280a95f-4268-45d5-be10-30dff4d1c810. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 내용을 확인해보실래요?\n\n상품: 백설기\n수량: 2개\n가격: 13,000원\n\n주문 변경을 원하시나요? 최종 확인 부탁드립니다.', response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 185, 'total_tokens': 251}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-215495d7-2c89-4565-8cdd-cdb7b4a3bbf0-0')